In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
pandas_profiling.ProfileReport(df_train)

Number of variables,12
Number of observations,891
Total Missing (%),8.1%
Total size in memory,83.6 KiB
Average record size in memory,96.1 B
Numeric,6
Categorical,4
Boolean,1
Date,0
Text (Unique),1
Rejected,0


Survived : 1が生存、0が死亡  
Pclass :　乗客の階級。１が高く、３が低い 。欠損値はない。
SibSp : 乗船していた兄弟または配偶者の数  
parch : 乗船していた親または子どもの数  
ticket : チケット番号  
cabin : 客室番号  
embarked : 乗船した港   C = Cherbourg   Q = Queenstown  S = Southampton 

In [3]:
full_data = [df_train, df_test]

### Pclass

In [4]:
print(df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


### Sexについて

In [5]:
print(df_train[['Sex','Survived']].groupby(['Sex'], as_index=False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


### Ageについて
中央値で埋めてみる。  
-> 年代のようにするために区切る

In [6]:
for data in full_data:
    data['Age'] = data['Age'].fillna(data['Age'].median())

df_train['punctuated_age'] = pd.cut(df_train['Age'], 4)

print(df_train[['punctuated_age','Survived']].groupby(['punctuated_age'], as_index=False).mean())

    punctuated_age  Survived
0   (0.34, 20.315]  0.458101
1  (20.315, 40.21]  0.364769
2  (40.21, 60.105]  0.390625
3   (60.105, 80.0]  0.227273


### SibSpとparchについて
家族の人数を作成。そこから生存の平均値をみる

In [7]:
for data in full_data:
    data['family_structure'] = data['SibSp'] + data['Parch'] + 1
print (df_train[['family_structure', 'Survived']].groupby(['family_structure'], as_index=False).mean())

   family_structure  Survived
0                 1  0.303538
1                 2  0.552795
2                 3  0.578431
3                 4  0.724138
4                 5  0.200000
5                 6  0.136364
6                 7  0.333333
7                 8  0.000000
8                11  0.000000


### embarkedについて
欠損が2しかないので、とりあえずもっとよく出てくるSouthamptonで埋める。そしてダミー変数化

In [8]:
for data in full_data:
    data['Embarked'] = data['Embarked'].fillna('S')
    
print(df_train[['Embarked','Survived']].groupby(['Embarked'], as_index=False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


### fareについて
運賃の幅に応じての生存率をみる　 　　　

→ ４rangeに分ける

In [9]:
for data in full_data:
    data['Fare'] = data['Fare'].fillna(df_train['Fare'].median())

df_train['Fare_range'] = pd.qcut(df_train['Fare'], 4)
print(df_train[['Fare_range','Survived']].groupby('Fare_range',as_index=False).mean())

        Fare_range  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


### Ticketについて
無視

### Cabinについて
無視

### Nameについて
何に使えるか不明なので、とりあえず敬称を抜き出して、Nameと関係がありそうな変数とクロス集計してみる    
→ 年齢と性別を予測するのに使えそう。    
一旦放置

In [11]:
print(pd.__version__)

0.22.0


### 前処理

In [10]:
for data in full_data:
    #sex convert to dummy variable
    data['Sex'] = pd.get_dummies(data['Sex'], drop_first=True)
    
    #Age_range
    data['Age_range'] = ''
    data['Age_range'][data['Age'] <= 22.0] = '1'
    data['Age_range'][(data['Age'] > 20.315) & (data['Age'] <= 40.21)] = '2'
    data['Age_range'][(data['Age'] > 40.21) & (data['Age'] <= 60.105)] = '3'
    data['Age_range'][data['Age'] > 60.105] = '4'
    
    #embarked convert to dummy variable
    data['Embarked'] = pd.get_dummies(data['Embarked'],drop_first=True)
    
    #fare_range convert to dummy variable
    data['fare_range'] = ''
    data['fare_range'][data['Fare'] <= 7.91] = '1'
    data['fare_range'][(data['Fare'] > 7.91) & (data['Fare'] <= 14.454)] = '2'
    data['fare_range'][(data['Fare'] > 14.454) & (data['Fare'] <= 31.0)] = '3'
    data['fare_range'][data['Fare'] > 31.0] = '4'

/Users/ryo.tanaka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/ryo.tanaka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ryo.tanaka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/ryo.tanaka/anaconda3/lib/python3.6/site-packages/ipykernel_laun

In [13]:
df_train = df_train.drop(['PassengerId','Name','Age','SibSp','Parch','Ticket','Fare','punctuated_age'], axis=1)
df_train.head()

,Survived,Pclass,Sex,Cabin,Embarked,family_structure,Fare_range,Age_range,fare_range
0,0,3,1,NaN,0,2,"(-0.001, 7.91]",2,1
1,1,1,0,C85,0,2,"(31.0, 512.329]",2,4
2,1,3,0,NaN,0,1,"(7.91, 14.454]",2,2
3,1,1,0,C123,0,2,"(31.0, 512.329]",2,4
4,0,3,1,NaN,0,1,"(7.91, 14.454]",2,2


In [14]:
df_train = df_train.drop(['Cabin','Fare_range'], axis=1)
df_train.head()

,Survived,Pclass,Sex,Embarked,family_structure,Age_range,fare_range
0,0,3,1,0,2,2,1
1,1,1,0,0,2,2,4
2,1,3,0,0,1,2,2
3,1,1,0,0,2,2,4
4,0,3,1,0,1,2,2


### ランダムフォレスト

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#separate explanatory variable and purpose variable
x = df_train.iloc[:, 1:]
y = df_train.iloc[:, [0]]

#separate train data and test data
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42)

#learning
clf = RandomForestClassifier(random_state=0)
clf = clf.fit(train_x,train_y)

/Users/ryo.tanaka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  del sys.path[0]


### 結果発表！！！

In [16]:
print('train_score:{}'.format(clf.score(train_x, train_y)))
print('test_score:{}'.format(clf.score(test_x, test_y)))

train_score:0.8667736757624398
test_score:0.8134328358208955


In [23]:
#get submit data
#drop_columns = ['Name','Age','SibSp','Parch','Ticket','Fare','Cabin']
#df_test = df_test.drop(drop_columns,axis=1)
df_test.head()

,PassengerId,Pclass,Sex,Embarked,family_structure,Age_range,fare_range
0,892,3,1,1,1,2,1
1,893,3,0,0,2,3,1
2,894,2,1,1,1,4,2
3,895,3,1,0,1,2,2
4,896,3,0,0,3,2,2


In [38]:
submit_feature = df_test[['Pclass','Sex','Embarked','family_structure','Age_range','fare_range']].values

pred = pd.Series(clf.predict(submit_feature), name='Survived').to_frame()
pred.head()

,Survived
0,0
1,0
2,0
3,0
4,0


In [44]:
submit_data = pd.concat([df_test[['PassengerId']], pred],axis=1)
print(submit_data.shape)
print(df_test.shape)

(418, 2)
(418, 7)


In [46]:
# to csv
submit_data.to_csv('titanic_sumit_data_rf.csv', index=False)

In [51]:
# 効いてる変数を順に並べる
importances_df = pd\
    .DataFrame(
        {'variable': ['Pclass','Sex','Embarked','family_structure','Age_range','fare_range'], 
         'importance': clf.feature_importances_})\
    .sort_values('importance', ascending=False).reset_index(drop=True)

display(importances_df)

,importance,variable
0,0.420756,Sex
1,0.156793,Pclass
2,0.144189,family_structure
3,0.131609,fare_range
4,0.123535,Age_range
5,0.023118,Embarked
